In [ ]:
import os
import yaml
import io
from jinja2 import Environment, FileSystemLoader
from snowflake.snowpark.context import get_active_session

def load_yaml_file(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)

def create_data_product_teams_dict(data_product_teams_path):
    data_dict = {}
    
    for subfolder in os.listdir(data_product_teams_path):
        subfolder_path = os.path.join(data_product_teams_path, subfolder)
        
        if os.path.isdir(subfolder_path):
            data_dict[subfolder] = {}
            
            data_products_file = os.path.join(subfolder_path, 'data-products.yaml')
            data_sources_file = os.path.join(subfolder_path, 'data-sources.yaml')
            
            if os.path.exists(data_products_file):
                data_dict[subfolder]['data-products'] = load_yaml_file(data_products_file)
            
            if os.path.exists(data_sources_file):
                data_dict[subfolder]['data-sources'] = load_yaml_file(data_sources_file)
    
    return data_dict

def main():
    data_product_teams_path = 'DATA-PRODUCT-TEAMS'
    dp_deployment_path = 'DP_DEPLOYMENT'
    
    data_product_teams_dict = create_data_product_teams_dict(data_product_teams_path)

    env = Environment(loader=FileSystemLoader(dp_deployment_path))
    data_product_team_template = env.get_template('data_product_team.sql.jinja')
    data_product_template = env.get_template('data_product.sql.jinja')
    data_source_template = env.get_template('data_source.sql.jinja')

    session = get_active_session()
    
    for team_name, team_data in data_product_teams_dict.items():
        data_product_team_sql = data_product_team_template.render(data_product_team=team_name)
        print(session.file.put_stream(io.BytesIO(data_product_team_sql.encode('utf-8')), \
                                    f'platform_admin.scripts.{team_name}/create_data_product_team_{team_name}.sql', \
                                    auto_compress=False,
                                    overwrite=True))
        
        for data_product in team_data['data-products']:
            data_product_sql = data_product_template.render(data_product_team=team_name, data_product=data_product)
            print(session.file.put_stream(io.BytesIO(data_product_sql.encode('utf-8')), \
                                        f'platform_admin.scripts.{team_name}/create_data_product_{team_name}_{data_product}.sql', \
                                        auto_compress=False,
                                        overwrite=True))

        for data_source in team_data['data-sources']:
            data_source_sql = data_source_template.render(data_product_team=team_name, data_source=data_source)
            print(session.file.put_stream(io.BytesIO(data_source_sql.encode('utf-8')), \
                                        f'platform_admin.scripts.{team_name}/create_data_sourcet_{team_name}_{data_source}.sql', \
                                        auto_compress=False,
                                        overwrite=True))

if __name__ == "__main__":
    main()
